In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time

In [2]:
# load dataset and labels
X_train = pickle.load(open('pickle_data/X_train.pickle','rb'))
y_train = pickle.load(open('pickle_data/y_train.pickle','rb'))

# load validation dataset and labels
X_validation = pickle.load(open('pickle_data/X_validation.pickle', 'rb'))
y_validation = pickle.load(open('pickle_data/y_validation.pickle', 'rb'))

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_validation.shape =', X_validation.shape)
print('y_validation.shape =', y_validation.shape)

X_train.shape = (2320, 50, 50, 3)
y_train.shape = (2320,)
X_validation.shape = (258, 50, 50, 3)
y_validation.shape = (258,)


In [3]:
# normalize pixel data
X_train = X_train/255.0
X_validation = X_validation/255.0

In [4]:
# define hyperparameters
conv_layer = 4
layer_size = 64
dense_layer = 0

In [5]:
# name of model to keep track of changing model architectures
NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
# create a new log of the model to analyze results
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# define model
model = Sequential()

# 1st convolution
model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# add additional conv layers
for _ in range(conv_layer-1):
    # convolution
    model.add(Conv2D(layer_size, (3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    # Max-Pool to reduce tensor size while retaining distict features
    model.add(MaxPooling2D(pool_size=(2,2)))

# flatten model for fully connected layers
model.add(Flatten())

# add dense layers
for _ in range(dense_layer):
    # fully-connected
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# final fully-connected layer (5 matches number of Pokemon categories)
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=20, validation_split=0.2, callbacks=[tensorboard])


Train on 1856 samples, validate on 464 samples
Epoch 1/20
1856/1856 [==============================] - 11s 6ms/sample - loss: 0.7965 - accuracy: 0.7258 - val_loss: 0.9768 - val_accuracy: 0.6810
Epoch 2/20
1856/1856 [==============================] - 10s 5ms/sample - loss: 0.4968 - accuracy: 0.8227 - val_loss: 0.8438 - val_accuracy: 0.7069
Epoch 3/20
1856/1856 [==============================] - 10s 5ms/sample - loss: 0.3953 - accuracy: 0.8680 - val_loss: 0.5271 - val_accuracy: 0.8276
Epoch 4/20
1856/1856 [==============================] - 10s 5ms/sample - loss: 0.3417 - accuracy: 0.8852 - val_loss: 0.9165 - val_accuracy: 0.6724
Epoch 5/20
1856/1856 [==============================] - 10s 6ms/sample - loss: 0.3057 - accuracy: 0.8933 - val_loss: 0.6184 - val_accuracy: 0.7888
Epoch 6/20
1856/1856 [==============================] - 10s 6ms/sample - loss: 0.2712 - accuracy: 0.8960 - val_loss: 1.6972 - val_accuracy: 0.5560
Epoch 7/20
1856/1856 [==============================] - 10s 6ms/sample 

In [6]:
# evaluate the model
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 67.83%


In [7]:
# save the model
model.save('models/' + NAME)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/4-conv-128-nodes-0-dense-1574901483/assets
